In [ ]:
import os
from glob import glob
from tqdm import tqdm
import datetime
import hail as hl

from hail.plot import show
import pandas as pd
from pprint import pprint
hl.plot.output_notebook()

from pyspark.conf import SparkConf
from pyspark.context import SparkContext
from pyspark.sql.session import SparkSession

#### Start an [Apache Spark](https://en.wikipedia.org/wiki/Apache_Spark) instance

In [ ]:
log_file_name = f"logs/hail-{datetime.datetime.now():%Y-%m-%d-%H-%M-%S}.log"
# run spark
spark_conf = SparkConf().setAppName("hail-test")
# .setMaster("spark://spark-master:7077")
spark_conf.set("spark.hadoop.fs.s3a.endpoint", "http://lifemap-minio:9000/")
spark_conf.set("spark.hadoop.fs.s3a.access.key", "root")
spark_conf.set("spark.hadoop.fs.s3a.secret.key", "passpass" )
spark_conf.set("spark.hadoop.fs.s3a.connection.ssl.enabled", "false")
spark_conf.set("spark.hadoop.fs.s3a.path.style.access", "true")
spark_conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark_conf.set("spark.hadoop.fs.s3a.connection.maximum", 1024);
spark_conf.set("spark.hadoop.fs.s3a.threads.max", 1024);
spark_conf.set("spark.hadoop.fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")

try:
    sc = SparkContext(conf=spark_conf)
except:
    print ("Spark session already up")

#### Create bucket on [Minio](https://min.io/) if it does not exists

In [ ]:
import boto3
from botocore.exceptions import NoCredentialsError

# S3 configuration
s3 = boto3.client(
    's3',
    endpoint_url="http://lifemap-minio:9000",
    aws_access_key_id="root",
    aws_secret_access_key="passpass",
)

bucket_name = "data-hail"

# Check if the bucket exists, if not, create it
try:
    s3.head_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' exists.")
except Exception:
    # If the bucket does not exist, create it
    s3.create_bucket(Bucket=bucket_name)
    print(f"Bucket '{bucket_name}' created.")

### [Hail](https://hail.is/) initialization

In [ ]:
hl.init(sc=sc, log=log_file_name)

#### Set filenames

In [ ]:
## VCF
vcf_fn = 'data/1kg.vcf'
## Annotation file
annotations_fn = 'data/1kg_annotations.txt'
## Matrix table
mt_fn = 's3://data-hail/1kg.mt'

print (f"VCF fn: {vcf_fn}")
print (f"Annotation file fn: {annotations_fn}")
print (f"Matrix table fn: {mt_fn}")

#### Reading vcf with Pandas (N/A if the vcf is stored on s3)

In [ ]:
vcf_pd = None
if "s3://" not in vcf_fn: 
    vcf_pd = pd.read_csv(vcf_fn, sep="\t", header=109, low_memory=False)
vcf_pd

### Import VCF to hail as Matrix Table

The VCF file must be converted into the [Hail Matrix Table](https://hail.is/docs/0.2/overview/matrix_table.html) data structure.
This structure builds upon the [Hail Table](https://hail.is/docs/0.2/hail.Table.html#hail.Table) and comprises four key components:

- a **two-dimensional matrix of entry fields** where each entry **is indexed by row key(s) and column key(s)**
- a corresponding **rows table** that stores all of the row fields that are constant for every column in the dataset
- a corresponding **columns table** that stores all of the column fields that are constant for every row in the dataset
- a set of **global fields** that are constant for every entry in the dataset

Every Hail Table has a **key** that controls both the order of the rows in the table and the ability to join or annotate one table with the information in another table. 
Matrix tables have **two keys**: a **row key** and a **column key**. Row fields are indexed by the row key, column fields are indexed by the column key, and entry fields are indexed by the row key and the column key.

In [ ]:
## Read a vcf file, convert and write it as matrix table
_ = hl.import_vcf(vcf_fn).write(mt_fn, overwrite=True) # assign this to a dummy variable to avoid errors

In [ ]:
## Read the matrix table from the file and assign it to the mt vaiable
mt = hl.read_matrix_table(mt_fn)
row_table = mt.rows()
col_table = mt.cols()
entry_fields = mt.entries()

print (f"Row table: {row_table}")
print (f"Col table: {col_table}")
print (f"Entries: {entry_fields}")

![alt text](immagini/vcf_matrix_table.png "Title")

In [ ]:
## Summary of the matrix table:
mt.describe()

#### Row table:

In [ ]:
row_table.describe()

In [ ]:
row_table.show()

#### Column table 

In [ ]:
col_table.describe()

In [ ]:
col_table.show()

#### Counting samples and variants

In [ ]:
## Counts of samples and variants
print (" Printing row and column fields with different methods")
# count row fields
print(mt.count_rows())
# same as
print (row_table.count())

# count column fields
print(mt.count_cols())
# same as
print(col_table.count())

n_variants = mt.count_rows() # This can be done accessing directy the row table with mt.rows().count()
n_samples = mt.count_cols()  # This can be done accessing directy the column table with mt.cols().count()

print (f"\n\nTable has {n_variants} variants and {n_samples} samples") 

#### Entry fields.

In [ ]:
entry_fields.describe()

In [ ]:
entry_fields.show(n_samples + 1)

In [ ]:
mt.GT.summarize()

#### Global values.
Common values of the matrix table

In [ ]:
mt.globals_table().show()

### Summary:  
In the Hail Matrix Table, VCF data are represented as follows:  

- **Row fields:** Each row corresponds to a variant, and row fields contain information common to that variant (e.g., alleles or quality).  
- **Column fields:** Each column represents a sample, with column fields specifying sample-related attributes such as the sample ID.  
- **Entry fields:** These contain data specific to a (variant, sample) pair, such as the genotype read for a particular sample.  
- **Column key:** The column field used for join operations.  
- **Row key:** The row field used for join operations.  

### Adding column fields to the matrix table starting from a metadata Table

Metadata of samples (like phenotypes or geographical origin) are saved in a separate txt file imported to Hail as a Table.

A Hail MatrixTable can have any number of row fields and column fields for storing data associated with each row and column. Annotations are usually a critical part of any genetic study. **Column fields are where you’ll store information about sample phenotypes, ancestry, sex, and covariates. Row fields can be used to store information like gene membership and functional impact for use in QC or analysis**.

We'll take the text file and use it to annotate the columns in the MatrixTable.

The annotation file contains the sample ID, the population and “super-population” designations, the sample sex, and two simulated phenotypes (one binary, one discrete).

This file can be imported into Hail with `import_table`. This function produces a **Table object**. This object functions similarly to a Pandas or R DataFrame but is not constrained by the memory of a single machine and it is distributed across Spark. **The Table object, like the matrix table object, is immutable**. To interact with it locally as a Python datastructure, you should use the `take` method or transform to a Pandas dataframe (using `to_pandas` method).

**Table can have global field and row fields**. Global field is common for each element of the table whereas row fields are specific for each row. In this case, each row refers specific attributes of a sample. For example, its ID is specified in the "Sample" row field whereas the other fields show its metadata. 

In [ ]:
annotation_table = (hl.import_table(annotations_fn, impute=True)
         .key_by('Sample'))

In [ ]:
annotation_table.describe()

In [ ]:
annotation_table.show(5)

#### Query functions and the [Hail Expression](https://hail.is/docs/0.2/overview/expressions.html) Language

Hail has a number of useful query functions that can be used for gathering statistics on our dataset. These query functions take Hail Expressions as arguments. Hail’s expressions are lazy representations of data (each data type in Hail has its own Expression class. For example, an Int32Expression represents a 32-bit integer, and a BooleanExpression represents a boolean value of True or False).

We will start by looking at some statistics of the information in our table. The [`aggregate`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.aggregate) method can be used to aggregate over rows of the table (see [Aggregation](https://hail.is/docs/0.2/guides/agg.html) and [Aggretators](https://hail.is/docs/0.2/aggregators.html#sec-aggregators) for details).

[`counter`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.counter) is an aggregation function that counts the number of occurrences of each unique element. We can use this to pull out the population distribution by passing in a Hail Expression for the field that we want to count by.

The aggregate method is then used to aggregate something in the table across different rows and the aggregate function like counter, stats, etc... are used to specify how and what to aggregate.

In [ ]:
## Population distribution
## Here counter counts unique geographycal origin label

aggregate_expression = hl.agg.counter(annotation_table.SuperPopulation)
pprint(annotation_table.aggregate(aggregate_expression))

[`stats`](https://hail.is/docs/0.2/aggregators.html#hail.expr.aggregators.stats) is an aggregation function that produces some useful statistics about numeric collections. We can use this to see the distribution of the CaffeineConsumption phenotype.

In [ ]:
## Stats perform some statistics on the specified field
## Here take stats of the caffeine consumption

aggregate_expression = hl.agg.stats(annotation_table.CaffeineConsumption)
pprint(annotation_table.aggregate(aggregate_expression))

The functionality demonstrated in the last few cells isn’t anything especially new: **it’s certainly not difficult to ask these questions with Pandas or R dataframes, or even Unix tools like awk. But Hail can use the same interfaces and query language to analyze collections that are much larger, like the set of variants.**

#### Grouping function to get some information within superpoulation
The Table [`group_by`](https://hail.is/docs/0.2/hail.Table.html#hail.Table.group_by) method can be used to apply aggregation functions to different groups.

In [ ]:
grp = annotation_table.group_by('SuperPopulation')

In [ ]:
grp.aggregate(stats=hl.agg.stats(annotation_table.CaffeineConsumption)).show()

#### Join sample annotations with the matrix table

Using the [`annotate_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.annotate_cols) method is possible to join the annotation table with the MatrixTable containing our dataset.
First, we’ll print the existing column schema using `col`. [`col`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.col) is an attribute of the matrix table that return struct expression of all column-indexed fields, including keys.
It is different from the [`cols()`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.cols) method that returns a table with all column fields in the matrix.

In [ ]:
#Column table before adding per sample annotation:
mt.col.describe()

In [ ]:
mt = mt.annotate_cols(pheno = annotation_table[mt.s])

# After the annotation the columns has a new field pheno,
# a struct that contains sample metadata
mt.col.describe()

Each column "name" now specifies the sample ID and its phenotype.  

In [ ]:
print(f"Metadata table samples: {annotation_table.count()}")
print(f"Matrix table samples: {mt.cols().count()}")

Since there are fewer samples in our dataset than in the full thousand genomes cohort, we need to look at annotations on the dataset. We can use [`aggregate_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.aggregate_cols) to get the metrics for only the samples in our dataset.

In [ ]:
mt.aggregate_cols(hl.agg.counter(mt.pheno.SuperPopulation))

The functionality demonstrated in the last few cells isn’t anything especially new: it’s certainly not difficult to ask these questions with **Pandas** or **R** dataframes, or even Unix tools like `awk`. But **Hail** can use the same interfaces and query language to analyze collections that are much larger, like the set of variants.

### Quality control

QC is where analysts spend most of their time with sequencing datasets. QC is an iterative process, and is different for every project: there is no “push-button” solution for QC. Each time the Broad collects a new group of samples, it finds new batch effects. However, by practicing open science and discussing the QC process and decisions with others, we can establish a set of best practices as a community.

QC is entirely based on the ability to understand the properties of a dataset. Hail attempts to make this easier by providing the [`sample_qc`](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.sample_qc) function, which produces a set of useful metrics and stores them in a column field.

In [ ]:
mt.col.describe()

In [ ]:
# sample_qc is a hail genetic method to compute per-sample metrics useful for quality control.
mt = hl.sample_qc(mt)
mt.col.describe()

In [ ]:
##Plotting the QC metrics is a good place to start.
p = hl.plot.histogram(mt.sample_qc.call_rate, range=(.88,1), legend='Call Rate')
show(p)

In [ ]:
## Checking corralations between the mean value of dp and the call rate
p = hl.plot.scatter(mt.sample_qc.dp_stats.mean, mt.sample_qc.call_rate, xlabel='Mean DP', ylabel='Call Rate')
p.line([2,22], [0.97,0.97], color='red', line_width=2)
p.line([4,4], [0.878,1.0], color='red', line_width=2)
show(p)

#### Removing outliers

Removing outliers from the dataset will generally improve association results. We can make arbitrary cutoffs and use them to filter.
Using matrix table [`filter_cols`](https://hail.is/docs/0.2/hail.MatrixTable.html#hail.MatrixTable.filter_cols) method it is possible to **create a new matrix table considering samples with the DP mean >= 4 and a call rate >= 0.97**. Samples that don't satisfy these criteria are removed.
The filtering method does not perform in-place filtering, so the result must be assigned to a variable for the changes to take effect.

In [ ]:
mt = mt.filter_cols((mt.sample_qc.dp_stats.mean >= 4) & (mt.sample_qc.call_rate >= 0.97))
print('After filter, %d/284 samples remain.' % mt.count_cols())

Next is genotype QC. It’s a good idea to filter out genotypes where the reads aren’t where they should be: if we find a genotype called homozygous reference with >10% alternate reads, a genotype called homozygous alternate with >10% reference reads, or a genotype called heterozygote without a ref / alt balance near 1:1, it is likely to be an error.

In a low-depth dataset like 1KG, it is hard to detect bad genotypes using this metric, since a read ratio of 1 alt to 10 reference can easily be explained by binomial sampling. However, in a high-depth dataset, a read ratio of 10:100 is a sure cause for concern!



In [ ]:
ab = mt.AD[1] / hl.sum(mt.AD)

filter_condition_ab = ((mt.GT.is_hom_ref() & (ab <= 0.1)) |
                        (mt.GT.is_het() & (ab >= 0.25) & (ab <= 0.75)) |
                        (mt.GT.is_hom_var() & (ab >= 0.9)))

fraction_filtered = mt.aggregate_entries(hl.agg.fraction(~filter_condition_ab))
print(f'Filtering {fraction_filtered * 100:.2f}% entries out of downstream analysis.')
mt = mt.filter_entries(filter_condition_ab)

Variant QC computes per per-variant metric useful for quality control. It is a bit more of the same of sample_qc: we can use the [`variant_qc`](https://hail.is/docs/0.2/methods/genetics.html#hail.methods.variant_qc) function to produce a variety of useful statistics, plot them, and filter. This is made at row level beacause they are stats on variants.

In [ ]:
mt = hl.variant_qc(mt)
mt.row.describe()

## Let’s do a GWAS!

First, we need to restrict to variants that are :

- common (we’ll use a cutoff of 1%)
- not so far from Hardy-Weinberg equilibrium as to suggest sequencing error

In [ ]:
mt = mt.filter_rows(mt.variant_qc.AF[1] > 0.01) # It takes variants for which the alternate allele has a frequency larger than 1%
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

mt = mt.filter_rows(mt.variant_qc.p_value_hwe > 1e-6) # Hardy-Weinberg equilibrium pvalue cut-off
print('Samples: %d  Variants: %d' % (mt.count_cols(), mt.count_rows()))

These filters removed about 15% of sites (we started with a bit over 10,000). This is NOT representative of most sequencing datasets! We have already downsampled the full thousand genomes dataset to include more common variants than we’d expect by chance.

In Hail, the association tests accept column fields for the sample phenotype and covariates. Since we’ve already got our phenotype of interest (caffeine consumption) in the dataset, we are good to go:

In [ ]:
gwas = hl.linear_regression_rows(y=mt.pheno.CaffeineConsumption,
                                 x=mt.GT.n_alt_alleles(),
                                 covariates=[1.0])
gwas.row.describe()

Looking at the bottom of the above printout, you can see the linear regression adds new row fields for the beta, standard error, t-statistic, and p-value.

Hail makes it easy to visualize results! Let’s make a Manhattan plot:

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

This doesn’t look like much of a skyline. Let’s check whether our GWAS was well controlled using a Q-Q (quantile-quantile) plot.

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

## Confounded!

The observed p-values drift away from the expectation immediately. Either every SNP in our dataset is causally linked to caffeine consumption (unlikely), or there’s a confounder.

We didn’t tell you, but sample ancestry was actually used to simulate this phenotype. This leads to a **stratified distribution** of the phenotype. The solution is to include ancestry as a covariate in our regression.

The **linear_regression_rows** function can also take column fields to use as covariates. We already annotated our samples with reported ancestry, but it is good to be skeptical of these labels due to human error. Genomes don’t have that problem! Instead of using reported ancestry, we will use genetic ancestry by including computed principal components in our model.

The **pca** function produces eigenvalues as a list and sample PCs as a Table, and can also produce variant loadings when asked. The **hwe_normalized_pca** function does the same, using HWE-normalized genotypes for the PCA.

In [ ]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt.GT)

In [ ]:
pprint(eigenvalues)

In [ ]:
pcs.show(5, width=100)

Now that we’ve got principal components per sample, we may as well plot them! Human history exerts a strong effect in genetic datasets. Even with a 50MB sequencing dataset, we can recover the major human populations.

In [ ]:
mt = mt.annotate_cols(scores = pcs[mt.s].scores)

In [ ]:
mt.describe()

In [ ]:
p = hl.plot.scatter(mt.scores[0],
                    mt.scores[1],
                    label=mt.pheno.SuperPopulation,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

Now we can rerun our linear regression, controlling for sample sex and the first few principal components. We’ll do this with input variable the number of alternate alleles as before, and again with input variable the genotype dosage derived from the PL field.

In [ ]:
gwas = hl.linear_regression_rows(
    y=mt.pheno.CaffeineConsumption,
    x=mt.GT.n_alt_alleles(),
    covariates=[1.0, mt.pheno.isFemale, mt.scores[0], mt.scores[1], mt.scores[2]])


In [ ]:
gwas.show()

We’ll first make a Q-Q plot to assess inflation…

In [ ]:
p = hl.plot.qq(gwas.p_value)
show(p)

That’s more like it! This shape is indicative of a well-controlled (but not especially well-powered) study. And now for the Manhattan plot:

In [ ]:
p = hl.plot.manhattan(gwas.p_value)
show(p)

We have found a caffeine consumption locus!

#### How to save the Matrix table

In [ ]:
mt_out_fn = 's3://data-hail/1kg_after_gwas.mt'
mt.write(mt_out_fn, overwrite=True)